In [1]:
import tensorflow as tf

2024-01-23 21:40:08.791675: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 21:40:08.812082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 21:40:08.812103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 21:40:08.812680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 21:40:08.816062: I tensorflow/core/platform/cpu_feature_guar

In [478]:
import numpy as np
import torch

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and reshape
train_images = train_images / 255.0
test_images = test_images / 255.0
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

# Convert to PyTorch format [batch_size, channels, height, width]
train_images_pt = torch.tensor(train_images).permute(0, 3, 1, 2).float()
test_images_pt = torch.tensor(test_images).permute(0, 3, 1, 2).float()

# If additional normalization is required for your PyTorch model, apply it here
# For example, if you use transforms.Normalize((0.1307,), (0.3081,)) in PyTorch, apply similar normalization
mean, std = 0.1307, 0.3081
train_images_pt = (train_images_pt - mean) / std
test_images_pt = (test_images_pt - mean) / std


In [468]:
# Define the LeNet model in TensorFlow
model_tf = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(84, activation='sigmoid'),
    tf.keras.layers.Dense(10)  # Assuming 10 classes
])

# Compile the model
model_tf.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

# Train the model
history = model_tf.fit(train_images, train_labels, epochs=3, batch_size=32, validation_split=0.1)



Epoch 1/3
1688/1688 [==============================] - 3s 2ms/step - loss: 0.8276 - accuracy: 0.7365 - val_loss: 0.2379 - val_accuracy: 0.9313
Epoch 2/3
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2291 - accuracy: 0.9300 - val_loss: 0.1407 - val_accuracy: 0.9608
Epoch 3/3
1688/1688 [==============================] - 3s 2ms/step - loss: 0.1550 - accuracy: 0.9533 - val_loss: 0.1077 - val_accuracy: 0.9685
313/313 - 0s - loss: 0.1234 - accuracy: 0.9614 - 209ms/epoch - 667us/step

Test accuracy: 0.9613999724388123


In [524]:
# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.1234 - accuracy: 0.9614 - 197ms/epoch - 629us/step

Test accuracy: 0.9613999724388123


In [664]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class LeNetPT(nn.Module):
    def __init__(self):
        super(LeNetPT, self).__init__()
        # Convolutional encoder
        self.conv1 = nn.Conv2d(1, 6, 5)  # 1 input channel, 6 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel

        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(256,120) # 256 * 120
        self.fc2 = nn.Linear(120, 84)         # 120 inputs, 84 outputs
        self.fc3 = nn.Linear(84, 10)          # 84 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # TODO: figure out the resize, currently work on batch_size = 1
        batch_size = x.size(0)
        x = x.reshape(x.size(0),-1).reshape(-1,16)  # Reshape to [batch_size, 16*4*4]
        x = np.transpose(x, (1,0))
        x = x.reshape(batch_size,-1)

        # Fully connected layers
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)  # No activation function here, will use CrossEntropyLoss later
        return x
    

model_pt = LeNetPT()

In [665]:
# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[6].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the third dense layer (fc3) from model_tf to model_pt
weights, biases = model_tf.layers[7].get_weights()
model_pt.fc3.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc3.bias = nn.Parameter(torch.from_numpy(biases))

In [666]:
# # Create a controlled input (e.g., an array of ones)
# controlled_input_tf = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
# controlled_input_pt = torch.from_numpy(controlled_input_tf).permute(0, 3, 1, 2)  # For PyTorch

# Select the image for TensorFlow
controlled_input_tf = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)



In [667]:
# Test PyTorch Basic Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)

output_tf = model_tf.predict(controlled_input_tf) 
print("TensorFlow Basic Model Output:", output_tf)
print("PyTorch Basic Model Output:", output_pt.cpu().numpy())

1/1 [==============================] - 0s 10ms/step
TensorFlow Basic Model Output: [[-2.4110148  -2.7046077   2.7064452   1.2696762  -4.830148   -2.670598
  -9.548576    6.460438   -2.2545085   0.18971601]]
PyTorch Basic Model Output: [[-2.4110146  -2.7046077   2.7064452   1.269676   -4.8301497  -2.6705985
  -9.548576    6.4604383  -2.254509    0.18971601]]


In [668]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming the TensorFlow MNIST data has already been loaded
# Convert test_images to PyTorch tensor and permute
test_images_pt = torch.tensor(test_images).permute(0, 3, 1, 2).float()

# Assuming test_labels are already loaded
test_dataset = TensorDataset(test_images_pt, torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model_pt, test_loader)
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 96.14%


In [520]:
4096/ 256

16.0

In [358]:
8192 / 32

256.0

## Layer 1

In [532]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)

    def forward(self, x):
        return torch.sigmoid(self.conv1(x))

model_pt_basic = BasicLeNetPT()

model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1))
])

In [533]:
import numpy as np
# Transfer weights for TensorFlow
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for PyTorch
with torch.no_grad():
    model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
    model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))


In [536]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)


In [537]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)

# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 14ms/step
TensorFlow Basic Model Output: [[[[0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   ...
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]]

  [[0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   ...
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]
   [0.3405072  0.37062928 0.6257144  0.33070144 0.6428052  0.62785596]]

  [[0.3405072  0.37062928 0.6257144  0.33070144

## Layer 2

In [538]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid')
])

In [539]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[1].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))


In [540]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)


In [541]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().permute(3,2,0,1).numpy())


1/1 [==============================] - 0s 21ms/step
TensorFlow Basic Model Output: [[[[2.57586502e-02 6.63225278e-02 3.52370068e-02 ... 2.83964783e-01
    3.78392786e-01 1.71559360e-02]
   [3.18357944e-02 9.97148827e-02 3.43728028e-02 ... 4.42336261e-01
    4.03742552e-01 2.45979186e-02]
   [4.90795746e-02 1.39458686e-01 3.03143449e-02 ... 5.38973391e-01
    4.56746191e-01 3.97939608e-02]
   ...
   [4.77694720e-02 6.36636093e-02 3.90103497e-02 ... 3.89785856e-01
    2.98766255e-01 4.16699834e-02]
   [3.69360596e-02 6.63337857e-02 3.34584303e-02 ... 2.79973477e-01
    3.29799414e-01 2.47441996e-02]
   [3.33412029e-02 6.84268177e-02 3.02623659e-02 ... 2.24366948e-01
    3.36898059e-01 1.78801026e-02]]

  [[5.97592928e-02 1.66600794e-01 2.22794805e-02 ... 6.77564323e-01
    2.82137245e-01 2.98533738e-02]
   [2.17131585e-01 3.99079263e-01 1.07417926e-02 ... 8.41790795e-01
    1.99286669e-01 6.10251874e-02]
   [6.20349586e-01 6.13339841e-01 3.73514532e-03 ... 8.61602783e-01
    9.75781530e-

## AvgPool + Flatten

In [627]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        # Flattening the tensor to the correct size
        #print ('x:', x.shape)
        #x = x.permute(3,2,0,1)
        x = x.reshape(x.size(0),-1).reshape(16,-1)  # Reshape to [batch_size, 16*4*4]
        x = np.transpose(x, (1,0))
        x = x.reshape(1,-1)
        #x = x.reshape(1,-1)

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten()
])

In [628]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt_basic
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))


In [629]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)

In [630]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().numpy())


1/1 [==============================] - 0s 41ms/step
TensorFlow Basic Model Output: [[5.48884988e-01 7.87806988e-01 5.87700084e-02 8.25578198e-02
  6.52495623e-01 8.31024814e-03 2.97849402e-02 4.15998220e-01
  5.69971740e-01 2.39701167e-01 9.86830413e-01 8.89949560e-01
  5.16457073e-02 7.28018768e-03 6.34472398e-03 8.25152849e-04
  4.76424456e-01 7.10247993e-01 4.32754517e-01 1.06224250e-02
  9.03416932e-01 3.78386199e-01 2.25163270e-02 4.04280424e-01
  5.69598019e-01 1.15150232e-02 9.22504008e-01 5.83597064e-01
  1.64265539e-02 5.89975063e-03 3.06279749e-01 1.18738087e-02
  4.98429477e-01 9.35147524e-01 2.74252802e-01 6.47156872e-03
  3.33572328e-01 1.86515957e-01 3.30121629e-02 5.46118207e-02
  4.28845465e-01 3.18194866e-01 9.84396935e-01 3.65795642e-02
  7.54113719e-02 2.42829397e-02 2.14797065e-01 8.91184807e-02
  4.86817956e-01 4.67130125e-01 3.00586456e-03 5.47560573e-01
  9.75016475e-01 7.92300142e-03 6.00912757e-02 2.13113964e-01
  2.77048886e-01 7.67062962e-01 9.48068202e-01 3.

## Dense Layer 1

In [631]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(256, 120) # 120 * 256

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool

        x = x.reshape(x.size(0),-1).reshape(16,-1)  # Reshape to [batch_size, 16*4*4]
        x = np.transpose(x, (1,0))
        x = x.reshape(1,-1)
        
        x = F.sigmoid(self.fc1(x))

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid')
])

In [632]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[5].get_weights()
model_tf_basic.layers[5].set_weights([weights, biases])


# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
print ('w:',weights)
model_pt_basic.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1,0))))
print ('w_T:',model_pt_basic.fc1.weight.shape)
model_pt_basic.fc1.bias = nn.Parameter(torch.from_numpy(biases))



w: [[ 0.0060004   0.09459859 -0.00982087 ...  0.4026808  -0.18616
  -0.0760118 ]
 [ 0.1680993   0.00600922  0.19005074 ...  0.16594718 -0.06219631
   0.05193466]
 [ 0.05982314 -0.2828379  -0.10398509 ... -0.2969587   0.3439928
   0.23956047]
 ...
 [ 0.14957191  0.01632433  0.04348282 ...  0.2801885  -0.09770978
   0.07889169]
 [-0.04528046 -0.11519922 -0.13521452 ...  0.35203415 -0.15684576
   0.07581601]
 [ 0.16932055  0.06789297 -0.24664897 ...  0.16121444  0.31199127
  -0.06370709]]
w_T: torch.Size([120, 256])


In [633]:
# Select the image for TensorFlow
controlled_input = test_images[36][np.newaxis, ]  # No reshape needed as it's already in (28, 28, 1) format
controlled_input_pt = torch.tensor(controlled_input_tf).float().permute(0, 3, 1, 2)

In [634]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().numpy())


1/1 [==============================] - 0s 42ms/step
TensorFlow Basic Model Output: [[2.5085372e-01 6.4726716e-01 4.8868883e-02 3.3194184e-01 2.9337928e-02
  1.0934564e-03 3.5817081e-01 9.5409507e-01 1.1309944e-01 1.6625345e-02
  3.0560943e-01 4.1540496e-02 9.8110253e-01 6.2015608e-02 2.9901081e-01
  8.9301074e-01 5.8131945e-01 9.0093009e-02 9.7839260e-01 2.0575029e-01
  1.0464395e-02 3.7924361e-01 9.9337846e-01 4.2498612e-01 2.8738566e-02
  1.1527718e-01 3.5135803e-01 1.6455298e-02 6.4079426e-02 2.0648109e-02
  3.4988154e-02 8.5316998e-01 1.3921203e-01 9.6092708e-03 9.7262537e-01
  9.8974645e-01 1.3170831e-01 9.6172369e-01 9.9841106e-01 1.6231669e-01
  3.6521304e-02 9.2579347e-01 7.3157918e-01 4.5203529e-02 7.4042553e-01
  3.7248302e-01 4.2575136e-01 9.8524725e-01 3.8875315e-02 5.3248071e-04
  8.7046909e-01 5.2938116e-01 5.6007969e-01 9.7692323e-01 8.4818944e-02
  8.3015889e-01 1.5019216e-01 1.9550809e-01 9.8865926e-01 9.3609399e-01
  7.3582703e-01 9.3234396e-01 9.8643106e-01 7.7345460

## Dense Layer 2

In [462]:
import torch
import torch.nn as nn
import tensorflow as tf
import numpy as np
import os

# Define a basic LeNet model in PyTorch with two convolutional layers
class BasicLeNetPT(nn.Module):
    def __init__(self):
        super(BasicLeNetPT, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layers / Dense block
        self.fc1 = nn.Linear(120, 256) # 256 * 120
        self.fc2 = nn.Linear(120,84)         # 120 inputs, 84 outputs

    def forward(self, x):
        # Convolutional block
        x = F.avg_pool2d(F.sigmoid(self.conv1(x)), (2, 2)) # Convolution -> Sigmoid -> Avg Pool
        x = F.avg_pool2d(F.sigmoid(self.conv2(x)), (2, 2)).permute(3,2,0,1) # Convolution -> Sigmoid -> Avg Pool

        # Flattening the tensor to the correct size
        x = x.reshape(x.size(0),-1).reshape(1,-1)  # Reshape to [batch_size, 16*4*4]
        
        # Print the shape of x before and after each layer
        print("Before fc1:", x.shape)
        
        x = F.sigmoid(self.fc1(x))
        
        print("After fc1:", x.shape)
        x = F.sigmoid(self.fc2(x))
        print("After fc2:", x.shape)

        return x

model_pt_basic = BasicLeNetPT()

# Create a basic LeNet model in TensorFlow with two convolutional layers
model_tf_basic = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='sigmoid', input_shape=(28, 28, 1)),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='sigmoid'),
    tf.keras.layers.AvgPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(84, activation='sigmoid')
])

In [463]:
# Transfer weights for the first Conv2D layer from the original model_tf
weights, biases = model_tf.layers[0].get_weights()
model_tf_basic.layers[0].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[2].get_weights()
model_tf_basic.layers[2].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[5].get_weights()
model_tf_basic.layers[5].set_weights([weights, biases])

# Transfer weights for the second Conv2D layer from the original model_tf
weights, biases = model_tf.layers[6].get_weights()
model_tf_basic.layers[6].set_weights([weights, biases])

# Transfer weights for the first Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt_basic.conv1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second Conv2D layer from model_tf to model_pt
weights, biases = model_tf.layers[2].get_weights()
model_pt_basic.conv2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (3, 2, 0, 1))))
model_pt_basic.conv2.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[5].get_weights()
model_pt_basic.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1,0))))
model_pt_basic.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[6].get_weights()
model_pt_basic.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1,0))))
model_pt_basic.fc2.bias = nn.Parameter(torch.from_numpy(biases))

In [464]:
# Create a controlled input (e.g., an array of ones)
controlled_input = np.ones((1, 28, 28, 1), dtype=np.float32)  # For TensorFlow
controlled_input_pt = torch.from_numpy(controlled_input).permute(0, 3, 1, 2)  # For PyTorch


In [466]:
# Test TensorFlow Basic Model
output_tf_basic = model_tf_basic.predict(controlled_input)

# Test PyTorch Basic Model
model_pt_basic.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt_basic = model_pt_basic(controlled_input_pt)


# Compare outputs
print("TensorFlow Basic Model Output:", output_tf_basic)
print("PyTorch Basic Model Output:", output_pt_basic.cpu().numpy())

1/1 [==============================] - 0s 9ms/step
Before fc1: torch.Size([1, 256])
After fc1: torch.Size([1, 120])
After fc2: torch.Size([1, 84])
TensorFlow Basic Model Output: [[0.7160134  0.8385422  0.0253077  0.42892176 0.6184138  0.9969837
  0.09171487 0.01182156 0.04299246 0.4810178  0.9516474  0.09384901
  0.8192958  0.55231595 0.10896716 0.00647116 0.04556889 0.8126056
  0.82992846 0.5289272  0.894237   0.08265778 0.9708183  0.24809329
  0.79544413 0.00390048 0.02803985 0.7674697  0.04256167 0.06272971
  0.3221807  0.43452185 0.6216037  0.04312156 0.97908485 0.72737664
  0.9310374  0.14843705 0.9657128  0.7040428  0.14954226 0.7755475
  0.2979573  0.5029429  0.85015994 0.19273664 0.04996477 0.02114206
  0.9176138  0.9626188  0.39863586 0.9899365  0.97770965 0.0713995
  0.02932361 0.17433517 0.99833655 0.8545451  0.11654612 0.21318336
  0.83583117 0.02372023 0.0214206  0.978412   0.852728   0.12194854
  0.23547943 0.55145293 0.06977712 0.49735156 0.98175037 0.28126732
  0.968428